# DELVE Hack Session 2025

The goal is to try and replicate the Dragonfly MRF algorithm and see if we can remove background sources to improve LSBG detection (maybe mdoeling?)

In [49]:
# basic packages
import numpy as np
import pandas as pd
import astropy.io.fits as pf
import fitsio as ft
import matplotlib.pyplot as plt
import glob

# image processing stuff
from astropy.stats import sigma_clipped_stats
from astropy.wcs import WCS
from astropy.stats import SigmaClip
from scipy import ndimage
from astropy.convolution import convolve, Gaussian2DKernel
import astropy.units as u
import mrf
from mrf.celestial import Celestial
from astropy.table import Table, Column, hstack
from mrf.display import display_single, SEG_CMAP

In [79]:
# glob it
tile_files = glob.glob('/Users/f006zcq/Desktop/LSST-LSBGs/DELVE-Hack25/images/*/DES2051-5540*.fits.fz')
cut_files = glob.glob('/Users/f006zcq/Desktop/LSST-LSBGs/DELVE-Hack25/images/*/cut*')
cat_file = glob.glob('/Users/f006zcq/Desktop/LSST-LSBGs/DELVE-Hack25/images/*/*coadd*')

In [81]:
# read in catalog and images
catalog = ft.read(cat_file[0])

g_hdu = pf.open(cut_files[0])
r_hdu = pf.open(cut_files[1])
i_hdu = pf.open(cut_files[2])

tile = pf.open(tile_files[0])['SCI'].header

In [82]:
# let's import a WCS from the images
wcs = WCS(g_hdu['SCI'].header)

In [83]:
# declare the thing we want to retain in the image
target_lsbg = 882985013
obj = catalog[catalog['COADD_OBJECT_ID']==target_lsbg]
print(obj['RA'],obj['DEC'])

[312.66972] [-55.608953]


In [87]:
head = g_hdu['SCI'].header
head['CD1_1'] = tile['CD1_1']
head['CD2_2'] = tile['CD2_2']


In [89]:
# now let's follow MRF's tutorial
mrf_g = Celestial(g_hdu['SCI'].data,header=head)

In [90]:
mrf_g.resize_image(0.5,method='cubic')
mrf_g.image = convolve(mrf_test.image,Gaussian2DKernel(1))
mrf_g.save_to_fits('mrf_test_df_g.fits')

In [91]:
mrf_r = Celestial(r_hdu['SCI'].data, header=head)
mrf_r.resize_image(0.5)
mrf_r.image = convolve(mrf_r.image, Gaussian2DKernel(1))
mrf_r.save_to_fits('mrf_test_df_r.fits')

In [94]:
task = MrfTask('decam_config.yaml')
img_lowres = 'mrf_test_df_g.fits'
img_hires_b = cut_files[0]
img_hires_r = cut_files[1]
certain_gal_cat = 'gal_cat_mrf.txt'

In [95]:
results = task.run(img_lowres, img_hires_b, img_hires_r, certain_gal_cat, 
                   output_name='mrf_test', verbose=True)

2025-06-27 15:23:13,077 Running Multi-Resolution Filtering (MRF) on "cfht" and "df" images!
2025-06-27 15:23:13,110 Magnify Dragonfly image with a factor of 3.0:


GalSimFFTSizeError: drawFFT requires an FFT that is too large.
The required FFT size would be 16384 x 16384, which requires 6.00 GB of memory.
If you can handle the large FFT, you may update gsparams.maximum_fft_size.